# Module 10 Application

## Challenge: Crypto Clustering

In this Challenge, you’ll combine your financial Python programming skills with the new unsupervised learning skills that you acquired in this module.

The CSV file provided for this challenge contains price change data of cryptocurrencies in different periods.

The steps for this challenge are broken out into the following sections:

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Find the Best Value for `k` Using the Original Data
* Cluster Cryptocurrencies with K-means Using the Original Data
* Optimize Clusters with Principal Component Analysis
* Find the Best Value for `k` Using the PCA Data
* Cluster the Cryptocurrencies with K-means Using the PCA Data
* Visualize and Compare the Results

### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [31]:
# Import required libraries and dependencies
import hvplot.pandas
import pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [32]:
# Load the data into a Pandas DataFrame
crypto = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

crypto

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [33]:
# Plot your data to see what's in your DataFrame
crypto.hvplot.scatter(
    width=1200,
    height=800,
    rot=60,
    xlabel="Cryptocurrency",
    ylabel="Price Changes"
)


:NdOverlay   [Variable]
   :Scatter   [coin_id]   (value)

In [34]:
# Generate summary statistics
crypto.describe()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [35]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data
scaled=StandardScaler().fit_transform(crypto)


# Create a DataFrame with the scaled data
model_scaled = pd.DataFrame(
    scaled,
    columns=crypto.columns
)

# Copy the crypto names from the original data
model_scaled["coin_id"]=crypto.index

# Set the coinid column as index
model_scaled = model_scaled.set_index("coin_id")

# Display sample data
model_scaled.head(10)


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351


### Find the Best Value for k Using the Original Data

In this section, you will use the elbow method to find the best value for `k`.

1. Code the elbow method algorithm to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following question: What is the best value for `k`?

In [36]:
# Create an empy list to store the inertia values
# Create a list with the number of k-values to try
# Use a range from 1 to 11
# Readjust
k = list(range(1, 11))

inertia = []

for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(model_scaled)
    inertia.append(k_model.inertia_)
    
elbow={"k": k, "inertia": inertia}
df = pd.DataFrame(elbow)


elbow_plot = df.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)
elbow_plot


C:\Users\Temp\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

#### Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

# **Answer:**  
 - 4 clusters appear to be the optimal number of clusters.

### Cluster Cryptocurrencies with K-means Using the Original Data

In this section, you will use the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the original data.

3. Predict the clusters to group the cryptocurrencies using the original data. View the resulting array of cluster values.

4. Create a copy of the original data and add a new column with the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [37]:

sub = KMeans(n_clusters=3, random_state=1)
sub.fit(model_scaled)
good=sub.predict(model_scaled)
good_pred=model_scaled.copy()
#sub was meant to stand for sub otpimal model and obviously optimal the opposite.
optimal = KMeans(n_clusters=4, random_state=1)
optimal.fit(model_scaled)
great = optimal.predict(model_scaled)
great_pred= model_scaled.copy()
#Fit models to scaled data




# View the resulting array of cluster values.
# Create a copy of the DataFrame



# Add a new column to the DataFrame with the predicted clusters

great_pred["CurrencyCluster"] = great
good_pred["CurrencyCluster"] = good

# Display sample data

Great_pred = great_pred.hvplot.scatter(
    x="price_change_percentage_200d",
    y="price_change_percentage_60d",
    by="CurrencyCluster",
    hover_cols="coin_id"
)


Good_pred = good_pred.hvplot.scatter(
    title="60days/200days",
    x="price_change_percentage_200d",
    y="price_change_percentage_60d",
    by="CurrencyCluster",
    hover_cols="coin_id"
)

Good_current = good_pred.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="CurrencyCluster",
    hover_cols="coin_id"
)

Great_current= great_pred.hvplot.scatter(
      title="7day/24hour",
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="CurrencyCluster",
    hover_cols="coin_id"
)
#Just having a bit of data science fun
Predictions=Good_pred*Great_pred
Current=Great_current*Good_current
display(Predictions+Current)

:Layout
   .Overlay.I  :Overlay
      .NdOverlay.I  :NdOverlay   [CurrencyCluster]
         :Scatter   [price_change_percentage_200d]   (price_change_percentage_60d,coin_id)
      .NdOverlay.II :NdOverlay   [CurrencyCluster]
         :Scatter   [price_change_percentage_200d]   (price_change_percentage_60d,coin_id)
   .Overlay.II :Overlay
      .NdOverlay.I  :NdOverlay   [CurrencyCluster]
         :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
      .NdOverlay.II :NdOverlay   [CurrencyCluster]
         :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

### Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [57]:
pca=PCA(n_components=3)

# Use the PCA model with `fit_transform` to reduce to
# three principal components.



crypto_pca=pca.fit_transform(model_scaled)

# View the first five rows of the DataFrame.
# Retrieve the explained variance to determine how much info  rmation
# can be attributed to each principal component. 
crypto_pca[:5]

pca.explained_variance_ratio_
sum=(.3720+0.3470+0.1760)

importance_of_pca=(sum*100)
importance_of_pca

89.5

#### Answer the following question: What is the total explained variance of the three principal components?

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89.5

In [58]:
DF_pca=pd.DataFrame(
    crypto_pca,
    columns=["PC1","PC2","PC3"]
)

DF_pca["coin_id"]=crypto.index

DF_pca=DF_pca.set_index("coin_id")

DF_pca.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


In [ ]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
# YOUR CODE HERE!
inertia_pca=[]
k_pca = list(range(1, 11))

for i in k_pca:
    k_pca_m= KMeans(n_clusters=i, random_state=1)
    k_pca_m.fit(DF_pca)
    inertia_pca.append(k_pca_m.inertia_)

# Create a dictionary with the data to plot the Elbow curve
PCA= {"k": k_pca, "inertia": inertia_pca}
elbow=pd.DataFrame(PCA)
# Create a DataFrame with the data to plot the Elbow curve
#df_pca_elbow
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
pca_elbow_plot = elbow.hvplot.line(
    x="k",
    y="inertia",
    title="PCA Elbow Curve",
    xticks=k
)

pca_elbow_plot

### Cluster Cryptocurrencies with K-means Using the PCA Data

In this section, you will use the PCA data and the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the principal components.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the PCA data.

3. Predict the clusters to group the cryptocurrencies using the PCA data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the PCA data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="PC1"` and `y="PC2"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [24]:
# Initialize the K-Means model using the best value for k
model_pca = KMeans(n_clusters=4, random_state=1)

model_pca.fit(DF_pca)

pca_clusters = model_pca.predict(DF_pca)

# View the resulting array of cluster values.
pca_clusters



array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 3, 0, 0, 0, 0])

In [25]:

pca_prediction_df =DF_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_prediction_df["PC_Currency_Cluster"] = pca_clusters

# Display sample data
pca_prediction_df

# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
pca_prediction_plot = pca_prediction_df.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="PC_Currency_Cluster"
)

pca_prediction_plot

:NdOverlay   [PC_Currency_Cluster]
   :Scatter   [PC1]   (PC2)

---

### Find the Best Value for k Using the PCA Data

In this section, you will use the elbow method to find the best value for `k` using the PCA data.

1. Code the elbow method algorithm and use the PCA data to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

In [26]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
# YOUR CODE HERE!
elbow_plot + pca_elbow_plot

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

#### Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?
* **Question:** What is the best value for `k` when using the PCA data?


  - **Answer:**  Four seems to be the optimal number of clusters.


* **Question:** Does it differ from the best k value found using the original data?


  -  **Answer:**  Only Slightly it is PCA Elbow Curve has a tighter inertia when compared with the Elbow Curve.

---

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

1. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the Elbow Curve that you created to find the best value for `k` with the original and the PCA data.

2. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the cryptocurrencies clusters using the original and the PCA data.

3. Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

> **Rewind:** Back in Lesson 3 of Module 6, you learned how to create composite plots. You can look at that lesson to review how to make these plots; also, you can check [the hvPlot documentation](https://holoviz.org/tutorial/Composing_Plots.html).

In [60]:
#We compare The Great Prediction graph to the PCA graph
Great_pred + pca_prediction_plot

:Layout
   .NdOverlay.I  :NdOverlay   [CurrencyCluster]
      :Scatter   [price_change_percentage_200d]   (price_change_percentage_60d,coin_id)
   .NdOverlay.II :NdOverlay   [PC_Currency_Cluster]
      :Scatter   [PC1]   (PC2)

#### Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 
  - Using less data did not change the optimal number of k or clusters. By using PCA method as a comparison between the two plots showed us similar outliers as well as a tighter and clearer inertia cluster within our PCA model.